In [1]:
import pandas as pd
import numpy as np
import requests
import glob
import openpyxl
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import helperfunctions as hf

Creating relative paths

In [2]:
votes = os.path.join("VoteWatch-EP-voting-data_2004-2022", "EP7_RCVs_2014_06_19.xlsx")
votations = os.path.join("VoteWatch-EP-voting-data_2004-2022","EP7_Voted docs.xlsx")

In [3]:
EP7_votes = pd.read_excel(votes)
EP7_votations = pd.read_excel(votations)


Importing data from EU API (for correct names)

In [4]:
url = 'https://data.europarl.europa.eu/api/v2/meps?parliamentary-term=7&format=application%2Fld%2Bjson&offset=0'
response = requests.get(url)
meps_data_7 = response.json()

In [5]:
meps_df_7 = pd.json_normalize(meps_data_7['data'])


MepID is incorrect in the original database, so I match MEP's by name from the API data

In [6]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [7]:
fuzzy_merged =fuzzy_merge(EP7_votes, meps_df_7, 'FullName', 'label', threshold=70, limit=1)

In [8]:
fuzzy_merged = fuzzy_merged.rename(columns={'matches': 'label'})

In [9]:

megamerge1 = fuzzy_merged.merge(meps_df_7,on='label')



In [10]:
boolmask = megamerge1.Lname.str.lower() == megamerge1.familyName.str.lower()

And then manually fix edge cases (the fuzzy_merge function prefers first name for some reason, 

In [11]:
name_to_new_id = {
    'ITURGAIZ ANGULO': '28398',
    'JONG': '96748',
    'MATO ADROVER': '96936'
}

for name, new_id in name_to_new_id.items():
    megamerge1.loc[megamerge1['Lname'] == name, 'identifier'] = new_id

Then merge again to get the correct familyName etc for the 3 meps (the following part is messy, I should've dropped most of the columns from meps_df_7 before proceeding, but it works so i decided to leave it in)

In [12]:
megamerge2 = megamerge1.merge(meps_df_7,on='identifier')


In [13]:
list_columns = megamerge2.columns.tolist()

filtered_list = [item for item in list_columns if isinstance(item, str)]
filtered_list

['MEP ID',
 'Fname',
 'Lname',
 'FullName',
 'Country',
 'Party',
 'EPG',
 'Start',
 'End',
 'label_x',
 'id_x',
 'type_x',
 'identifier',
 'familyName_x',
 'givenName_x',
 'sortLabel_x',
 'officialFamilyName_x',
 'officialGivenName_x',
 'id_y',
 'type_y',
 'label_y',
 'familyName_y',
 'givenName_y',
 'sortLabel_y',
 'officialFamilyName_y',
 'officialGivenName_y']

deleting redundant columns

In [14]:
Clean_columns = megamerge2.drop(columns=['MEP ID',
 'Fname',
 'Lname',
 'FullName',
 'id_x',
 'type_x',
 'officialFamilyName_x',
 'officialGivenName_x',
 'id_y',
 'type_y',
 'label_y',
 'familyName_y',
 'givenName_y',
 'sortLabel_y',
                                         'sortLabel_x',
 'officialFamilyName_y',
 'officialGivenName_y',])


In [15]:
Clean_columns.columns

Index([     'Country',        'Party',          'EPG',        'Start',
                'End',              1,              2,              3,
                    4,              5,
       ...
                 6958,           6959,           6960,           6961,
                 6962,           6963,      'label_x',   'identifier',
       'familyName_x',  'givenName_x'],
      dtype='object', length=6970)

Adding the missing Activ column

In [16]:
Clean_columns['Activ'] = ((Clean_columns['Start'] == '14/07/2009') & (Clean_columns['End'] == '31/12/2014')).map({True: 'yes', False: 'no'})

In [17]:
Clean_columns.Activ.value_counts()

Activ
yes    650
no     203
Name: count, dtype: int64

In [18]:
Clean_columns.columns


Index([     'Country',        'Party',          'EPG',        'Start',
                'End',              1,              2,              3,
                    4,              5,
       ...
                 6959,           6960,           6961,           6962,
                 6963,      'label_x',   'identifier', 'familyName_x',
        'givenName_x',        'Activ'],
      dtype='object', length=6971)

Reordering the columns

In [19]:
desired_order = [
    'identifier', 'givenName_x', 'familyName_x', 'label_x', 'Activ',
    'Country', 'Party', 'EPG', 'Start', 'End'
]

remaining_columns = [col for col in Clean_columns.columns if col not in desired_order]

final_column_order = desired_order + remaining_columns

Clean_columns_good_order = Clean_columns[final_column_order]



In [20]:
Clean_columns_good_order.columns

Index([  'identifier',  'givenName_x', 'familyName_x',      'label_x',
              'Activ',      'Country',        'Party',          'EPG',
              'Start',          'End',
       ...
                 6954,           6955,           6956,           6957,
                 6958,           6959,           6960,           6961,
                 6962,           6963],
      dtype='object', length=6971)

Renaming the columns

In [21]:
rename_dict = {
    'identifier': 'MepId',
    'givenName_x': 'Fname',
    'familyName_x': 'Lname',
    'label_x': 'FullName'
}
Final_df = Clean_columns_good_order.rename(columns=rename_dict)

Inspecting votations

In [22]:
EP7_votations.shape

(6961, 28)

In [23]:
EP7_votations.columns

Index(['Vote ID', 'File', 'Order of vote', 'Date', 'O'clock', 'Title',
       'Procedure', 'Leg/Non-Leg/Bud', 'Type of Vote', 'Voting Rule',
       'Rapporteur', 'Code', 'interinstitutional file number', 'Link',
       'Committee responsabile', 'De', 'Subject', 'Final vote?', 'Am No.',
       'Author', 'RCV', 'Vote', 'Yeas', 'No', 'Abs', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27'],
      dtype='object')

Dropping irrelevant columns

In [24]:
EP7_votations_columnsdropped = EP7_votations.drop(columns=['File', 'Order of vote', "O'clock",
        'Code', 'interinstitutional file number',
    
       'RCV',  'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27'])
EP7_votations_columnsdropped

,Vote ID,Date,Title,Procedure,Leg/Non-Leg/Bud,Type of Vote,Voting Rule,Rapporteur,Link,Committee responsabile,De,Subject,Final vote?,Am No.,Author,Vote,Yeas,No,Abs
0,1,14.09.2009,Election of the President of the European Comm...,NaN,Non,Agenda setting,s,0,http://www.europarl.europa.eu/activities/plena...,NaN,Internal regulations of the EP,0,1.0,NaN,0,-,135,248,14
1,2,15.09.2009,Agreement EC/Mongolia on certain aspects of ai...,*,Leg,Legislative resolution,s,Brian SIMPSON,http://www.europarl.europa.eu/sides/getDoc.do?...,Committee on Transport and Tourism,Transport & tourism,Single vote,1.0,NaN,0,+,456,21,2
2,3,15.09.2009,EC-China agreement: maritime transport operati...,*,Leg,Legislative resolution,s,Brian SIMPSON,http://www.europarl.europa.eu/sides/getDoc.do?...,Committee on Transport and Tourism,Transport & tourism,Single vote,1.0,NaN,0,+,558,19,11
3,4,15.09.2009,Mobilisation of the European Globalisation Adj...,NaN,Bud,Motions for a resolution,q,Reimer BÖGE,http://www.europarl.europa.eu/sides/getDoc.do?...,Committee on Budgets,Budget,Single vote,1.0,NaN,0,+,581,23,6
4,5,15.09.2009,Mobilisation of the European Union Solidarity ...,NaN,Bud,Motions for a resolution,q,Reimer BÖGE,http://www.europarl.europa.eu/sides/getDoc.do?...,Committee on Budgets,Budget,Single vote,1.0,NaN,0,+,578,26,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6956,6959,17.04.2014,Commission follow-up to the 'TOP TEN' Consulta...,NaN,Non,Motion for resolution,s,0,http://www.europarl.europa.eu/sides/getDoc.do?...,0,"Industry, research & energy",After recital F,0.0,11,Greens/EFA,+,503,11,1
6957,6960,17.04.2014,Commission follow-up to the 'TOP TEN' Consulta...,NaN,Non,Motion for resolution,s,0,http://www.europarl.europa.eu/sides/getDoc.do?...,0,"Industry, research & energy",Recital I,0.0,12,Greens/EFA,-,206,301,14
6958,6961,17.04.2014,Commission follow-up to the 'TOP TEN' Consulta...,NaN,Non,Motion for resolution,s,0,http://www.europarl.europa.eu/sides/getDoc.do?...,0,"Industry, research & energy",Recital J,0.0,13,Greens/EFA,-,198,301,21
6959,6962,17.04.2014,Commission follow-up to the 'TOP TEN' Consulta...,NaN,Non,Motion for resolution,s,0,http://www.europarl.europa.eu/sides/getDoc.do?...,0,"Industry, research & energy",vote: resolution (as a whole),1.0,NaN,0,+,311,184,15


Checking and fixing the datatypes

In [25]:
EP7_votations_columnsdropped['Date'] = pd.to_datetime(EP7_votations_columnsdropped['Date'], format='%d.%m.%Y')

Recoding columns in line with convention for the app

In [26]:
original_columns = EP7_votations_columnsdropped.columns.tolist()

new_columns = [
    "VoteId",
    "Date",
    "Title",
    "Procedure",
    "Leg/Non-Leg/Bud",
    "TypeOfVote",
    "VotingRule",
    "Rapporteur",
    "Link",
    "CommitteeResponsabile",
    "PolicyArea",
    "Subject",
    "FinalVote",
    "Author",
    "AmNo",
    "Vote",
    "Yes",
    "No",
    "Abs"
]

rename_dict = dict(zip(original_columns, new_columns))


EP7_votations_columnsdropped.rename(columns=rename_dict, inplace=True)


In [27]:
EP7_votations_columnsdropped.Vote.value_counts()

Vote
+    4763
-    2198
Name: count, dtype: int64

Recoding missing data

In [28]:
def recode_votations(df):
    df['TypeOfVote'] = df['TypeOfVote'].replace(0,np.NAN)
    df['VotingRule'] = df['VotingRule'].replace(0,np.NAN)
    df['Rapporteur'] = df['Rapporteur'].replace(0,np.NAN)
    df['CommitteeResponsabile'] = df['CommitteeResponsabile'].replace(0,np.NAN)
    df['Author'] = df['Author'].replace(0,np.NAN)
    df['Subject'] = df['Subject'].replace(0,np.NAN)
    df['AmNo'] = df['AmNo'].replace(0,np.NAN)
    df['Vote'] = df['Vote'].replace({'+': 1, '-': 0})


In [29]:
recode_votations(EP7_votations_columnsdropped)

C:\Users\zwiad\AppData\Local\Temp\ipykernel_4728\308808823.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Vote'] = df['Vote'].replace({'+': 1, '-': 0})


Checking and fixing the datatypes

In [30]:
EP7_votations_columnsdropped.dtypes

VoteId                            int64
Date                     datetime64[ns]
Title                            object
Procedure                        object
Leg/Non-Leg/Bud                  object
TypeOfVote                       object
VotingRule                       object
Rapporteur                       object
Link                             object
CommitteeResponsabile            object
PolicyArea                       object
Subject                          object
FinalVote                       float64
Author                           object
AmNo                             object
Vote                              int64
Yes                               int64
No                                int64
Abs                               int64
dtype: object

In [31]:
EP7_votations_columnsdropped.FinalVote = EP7_votations_columnsdropped['FinalVote'].astype('Int64')

Renaming EPG groups to fit the API convention


In [32]:
EPG_rename_dict = {
    "Group of the European People's Party (Christian Democrats)":"EPP",
    "Group of the Progressive Alliance of Socialists and Democrats in the European Parliament":"S&D",
    "Group of the Alliance of Liberals and Democrats for Europe":"ALDE",
    "European Conservatives and Reformists Group":"ECR",
    "Confederal Group of the European United Left - Nordic Green Left":"GUE–NGL",
    "Group of the Greens/European Free Alliance":"Greens–EFA",
    "Non-attached Members":"NI",  
    "Europe of freedom and democracy Group":"EFD"
}

In [33]:
Final_df.EPG = Final_df.EPG.replace(EPG_rename_dict, regex=False)

In [34]:
Final_df.columns


Index([   'MepId',    'Fname',    'Lname', 'FullName',    'Activ',  'Country',
          'Party',      'EPG',    'Start',      'End',
       ...
             6954,       6955,       6956,       6957,       6958,       6959,
             6960,       6961,       6962,       6963],
      dtype='object', length=6971)

Creating a list to filter the non,observation columns from Votes


In [35]:
filtered_list = ['MepId',
 'Fname',
 'Lname',
 'FullName',
 'Activ',
 'Country',
 'Party',
 'EPG',
 'Start',
 'End']

Melting votes into long format


In [36]:

mep_info = Final_df[filtered_list]
vote_columns = [col for col in Final_df.columns if col not in filtered_list]
melted_votes = pd.melt(Final_df, id_vars='MepId', value_vars=vote_columns, var_name='VoteId', value_name='Vote')


In [37]:
mep_info_for_wnominate = mep_info[['FullName','EPG']]
matrix_ep7_votes = Final_df[vote_columns]


In [38]:
epg_df = hf.get_epgs()
party_df = hf.get_parties()
ep_df = hf.generate_ep_df(9)
org_df = pd.concat([epg_df, party_df, ep_df], ignore_index=True)

In [39]:
##memberships_df = hf.get_memberships_df(mep_info,org_df)


In [40]:
##mep_info_db = hf.get_mep_database(mep_info,memberships_df)


In [41]:
votings_filtered_list = [
    "VoteId" 
    ,"Date" 
    ,"Title"
    ,"TypeOfVote"
    ,"Rapporteur"
    ,"Link"
    ,"CommitteeResponsabile"
    ,'Subject'
    ,'FinalVote' 
    ,'AmNo'
    ,'Author'
    ,'Vote'   
    ,'Yes'
    ,'No' 
    ,'Abs']

In [42]:
votings = EP7_votations_columnsdropped[votings_filtered_list]


In [43]:
melted_votes.VoteId = melted_votes.VoteId.astype('Int64')


In [44]:
melted_votes

,MepId,VoteId,Vote
0,96850,1,2
1,96659,1,2
2,96897,1,5
3,28367,1,2
4,96736,1,5
...,...,...,...
5937728,122885,6963,5
5937729,123792,6963,4
5937730,123816,6963,5
5937731,37200,6963,5


Saving files to .csv

In [45]:
base_directory = os.path.join("Cleaned_data","EP7_clean_data")
os.makedirs(base_directory, exist_ok=True)

In [46]:
matrix_ep7_votes.to_csv(os.path.join(base_directory,"matrix_ep7_votes.csv"),index=False)
mep_info_for_wnominate.to_csv(os.path.join(base_directory,"mep_info_for_wnominate.csv"),index=False)

In [47]:
# mep_info_db.to_csv(os.path.join(base_directory,"mep_info_EP_7.csv"),index=False)
# melted_votes.to_csv(os.path.join(base_directory,"votes_EP_7.csv"),index=False)
# votings.to_csv(os.path.join(base_directory,"votations_EP_7.csv"),index=False)
# memberships_df.to_csv(os.path.join(base_directory,"memberships_EP_7.csv"),index=False)